In [2]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import pandas as pd
import numpy as np
import gensim
import nltk
import re
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.corpus import stopwords
from pprint import pprint
from sklearn.datasets import fetch_20newsgroups
from nltk.stem.snowball import SnowballStemmer

2017-06-05 23:19:20,743 : INFO : 'pattern' package found; tag filters are available for English


In [3]:
newsgroups_train_meta = fetch_20newsgroups(subset='train')

## Categorias dos textos

In [4]:
pprint(list(newsgroups_train_meta.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


### Vamos ver um texto de Motos

In [5]:
newsgroups_train_meta.target_names[newsgroups_train_meta.target[10]]

'rec.motorcycles'

In [6]:
newsgroups_train_meta.data[10]

u'From: irwin@cmptrc.lonestar.org (Irwin Arnstein)\nSubject: Re: Recommendation on Duc\nSummary: What\'s it worth?\nDistribution: usa\nExpires: Sat, 1 May 1993 05:00:00 GMT\nOrganization: CompuTrac Inc., Richardson TX\nKeywords: Ducati, GTS, How much? \nLines: 13\n\nI have a line on a Ducati 900GTS 1978 model with 17k on the clock.  Runs\nvery well, paint is the bronze/brown/orange faded out, leaks a bit of oil\nand pops out of 1st with hard accel.  The shop will fix trans and oil \nleak.  They sold the bike to the 1 and only owner.  They want $3495, and\nI am thinking more like $3K.  Any opinions out there?  Please email me.\nThanks.  It would be a nice stable mate to the Beemer.  Then I\'ll get\na jap bike and call myself Axis Motors!\n\n-- \n-----------------------------------------------------------------------\n"Tuba" (Irwin)      "I honk therefore I am"     CompuTrac-Richardson,Tx\nirwin@cmptrc.lonestar.org    DoD #0826          (R75/6)\n------------------------------------------

### Agora um de computação

In [7]:
newsgroups_train_meta.target_names[newsgroups_train_meta.target[1]]

'comp.sys.mac.hardware'

In [8]:
newsgroups_train_meta.data[1]

u"From: guykuo@carson.u.washington.edu (Guy Kuo)\nSubject: SI Clock Poll - Final Call\nSummary: Final call for SI clock reports\nKeywords: SI,acceleration,clock,upgrade\nArticle-I.D.: shelley.1qvfo9INNc3s\nOrganization: University of Washington\nLines: 11\nNNTP-Posting-Host: carson.u.washington.edu\n\nA fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks.\n\nGuy Kuo <guykuo@u.washington.edu>\n"

### Remover os headers, footers e quotes dos textos (Metadados que não vão nos interessar hoje)

In [9]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

### Mesmo texto de computação

In [10]:
newsgroups_train.data[1]

u"A fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks."

Bem melhor o/

### Tokenização + Limpeza + Stopwords

Atenção! Cada problema que envolve texto exige um tratamento diferente nesta etapa. Ex. Caso o seu problema envolva análise de sentimento pode não ser uma boa ideia remover pontuação.

O pacote nltk tem bastante coisa de NLP que iremos aproveitar - tem algumas coisas para o português também!

In [11]:
def clean_and_tokenizing(text):
    # Remove numbers + punctuation        
    letters_only = re.sub("[^a-zA-Z]", " ", text) 

    # Convert to lower case, split into individual words
    words = letters_only.lower().split()                             

    # Save english stopwords
    stops = set(stopwords.words("english"))                  

    # Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    
    # Join everython again
    return( " ".join( meaningful_words ))   

In [12]:
num_text = len(newsgroups_train.data)

print "Cleaning and parsing...\n"
clean_texts = []
for i in xrange(0, num_text):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print "Review %d of %d\n" % ( i+1, num_text )
    clean_texts.append(clean_and_tokenizing(newsgroups_train["data"][i]))

Cleaning and parsing...

Review 1000 of 11314

Review 2000 of 11314

Review 3000 of 11314

Review 4000 of 11314

Review 5000 of 11314

Review 6000 of 11314

Review 7000 of 11314

Review 8000 of 11314

Review 9000 of 11314

Review 10000 of 11314

Review 11000 of 11314



In [13]:
clean_texts[1]

u'fair number brave souls upgraded si clock oscillator shared experiences poll please send brief message detailing experiences procedure top speed attained cpu rated speed add cards adapters heat sinks hour usage per day floppy disk functionality floppies especially requested summarizing next two days please add network knowledge base done clock upgrade answered poll thanks'

### Bag of Words

Primeiro vamos comparar as dimensões de um BoW com limpeza de um BoW sem limpeza:

#### Original

In [14]:
# Initialize the "CountVectorizer" is the bag of words

vectorizer = CountVectorizer(analyzer = "word")

train_data_features = vectorizer.fit_transform(newsgroups_train["data"])

train_data_features = train_data_features.toarray()

Perceba que o resultado tem 11314 documentos e 101631 palavras

In [15]:
print train_data_features.shape

(11314, 101631)


#### Clean

In [16]:
# Initialize the "CountVectorizer" is the bag of words

vectorizer = CountVectorizer(analyzer = "word")
# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer.fit_transform(clean_texts)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()

Perceba que o resultado tem 11314 documentos e 72772 palavras - Uma redução de 28%!

In [17]:
print train_data_features.shape

(11314, 72772)


In [18]:
feature_names = vectorizer.get_feature_names() 
feature_names[50:70]

[u'aavb',
 u'aavso',
 u'aaw',
 u'aax',
 u'aayau',
 u'aazo',
 u'ab',
 u'aba',
 u'ababs',
 u'abacus',
 u'abaihm',
 u'abandon',
 u'abandond',
 u'abandoned',
 u'abandoning',
 u'abandons',
 u'abatement',
 u'abates',
 u'abba',
 u'abbasids']

In [19]:
train_data_features[1]

array([0, 0, 0, ..., 0, 0, 0])

### Stemming

In [20]:
def stem(text):
    # Split into individual words
    words = text.split()                             

    # Save english stopwords
    snowball = SnowballStemmer("english")

    # Remove stop words
    stem_words = []
    for w in words:
        stem_words.append(snowball.stem(w))
    
    return( " ".join(stem_words))  

In [21]:
num_text = len(newsgroups_train.data)

print "Stemming...\n"
stem_texts = []
for i in xrange(0, num_text):
    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print "Review %d of %d\n" % ( i+1, num_text )
    stem_texts.append(stem(clean_texts[i]))

Stemming...

Review 1000 of 11314

Review 2000 of 11314

Review 3000 of 11314

Review 4000 of 11314

Review 5000 of 11314

Review 6000 of 11314

Review 7000 of 11314

Review 8000 of 11314

Review 9000 of 11314

Review 10000 of 11314

Review 11000 of 11314



#### New BoW with stem

In [22]:
# Initialize the "CountVectorizer" is the bag of words

vectorizer = CountVectorizer(analyzer = "word")
# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
stem_train_data_features = vectorizer.fit_transform(stem_texts)

# Numpy arrays are easy to work with, so convert the result to an 
# array
stem_train_data_array = stem_train_data_features.toarray()

Perceba que o resultado tem 11314 documentos e 55593 palavras - Uma redução de 54% do original! (limpeza + stemming)

In [23]:
print stem_train_data_features.shape

(11314, 55593)


In [24]:
feature_names = vectorizer.get_feature_names() 
feature_names[50:70]

[u'aaw',
 u'aax',
 u'aayau',
 u'aazo',
 u'ab',
 u'aba',
 u'abab',
 u'abacus',
 u'abaihm',
 u'abandon',
 u'abandond',
 u'abat',
 u'abb',
 u'abba',
 u'abbasid',
 u'abber',
 u'abbey',
 u'abbi',
 u'abbot',
 u'abbott']

In [39]:
motorcicle = newsgroups_train.target == 8

In [ ]:
motorcicle_docs = stem_train_data_features[motorcicle]

In [27]:
freqs = [(word, stem_train_data_features.getcol(idx).sum()) for word, idx in vectorizer.vocabulary_.items()]
#sort from largest to smallest
print sorted (freqs, key = lambda x: -x[1])

[(u'ax', 62485), (u'use', 7212), (u'one', 6781), (u'would', 6165), (u'max', 4680), (u'get', 4523), (u'like', 4498), (u'peopl', 4137), (u'know', 3926), (u'time', 3757), (u'think', 3400), (u'also', 3352), (u'say', 3056), (u'make', 3040), (u'go', 2959), (u'work', 2928), (u'year', 2866), (u'system', 2676), (u'file', 2635), (u'new', 2626), (u'good', 2552), (u'could', 2537), (u'well', 2537), (u'edu', 2437), (u'want', 2436), (u'may', 2435), (u'even', 2380), (u'see', 2365), (u'right', 2359), (u'look', 2358), (u'problem', 2316), (u'need', 2315), (u'thing', 2279), (u'way', 2266), (u'two', 2239), (u'first', 2238), (u'us', 2224), (u'program', 2177), (u'much', 2104), (u'mani', 2099), (u'tri', 2088), (u'god', 2071), (u'window', 2064), (u'call', 2030), (u'post', 2014), (u'point', 1877), (u'take', 1861), (u'come', 1856), (u'includ', 1848), (u'run', 1843), (u'question', 1769), (u'said', 1743), (u'state', 1743), (u'number', 1735), (u'believ', 1722), (u'mean', 1670), (u'drive', 1668), (u'key', 1667), (u'